# Running the `ialg` algorithm on the TSPLIB and the HardTSPLIB instances

Here, we want to compute the minimum number of SECs needed to prove optimality for some small famous instances in the TSPLIB [1] and some hard-to-solve instances in the HardTSPLIB [2]. We will use the `ialg` algorithm to compute the minimum number of SECs needed to prove optimality for these instances.

### References
[1] Reinelt, Gerhard. "TSPLIB—A traveling salesman problem library." ORSA journal on computing 3.4 (1991): 376-384.

[2] Vercesi, Eleonora, et al. "On the generation of metric TSP instances with a large integrality gap by branch-and-cut." Mathematical Programming Computation 15.2 (2023): 389-416.


In [23]:
from utils import ialg, from_tsplib_file_to_graph
import pandas as pd

## TSPLIB instances

These are the instances we were able to conduct the experiment on. Bigger instances were not feasible to run on our machine. If you want to test other instances, just add them to the `tsplib_instances` list at position 0.

In [24]:
tsplib_instances = [("burma14", 14), ("ulysses16", 16), ("gr17", 17), ("ulysses22", 22), ("fri26", 29), ("bayg29", 29), ("bays29", 29), ("swiss42", 42), ("dantzig42", 42)]

In [25]:
# Store the values in a dictionary
out = {}

Now, we run the `ialg` algorithm on the TSPLIB instances. Unfortunately, this may take a while

In [26]:
for instance_name, n in tsplib_instances:
    # Parse the instance
    G = from_tsplib_file_to_graph("./data/" + instance_name)
    print("******* Instance:", instance_name, "*******")
    S_family, S_num = ialg(G, verbose=True)
    out[instance_name] = S_num

******* Instance: burma14 *******
num_bb_nodes, num_subtour_constrs, num_conn_comp = 1 0 3
num_bb_nodes, num_subtour_constrs, num_conn_comp = 2 1 2
num_bb_nodes, num_subtour_constrs, num_conn_comp = 4 1 2
Found a solution with this many subtours: 2
Specifically, they are:
[0, 1, 7, 8, 9, 10]
[8, 9, 10]
S_family =  [[0, 1, 7, 8, 9, 10], [8, 9, 10]]
******* Instance: ulysses16 *******
num_bb_nodes, num_subtour_constrs, num_conn_comp = 1 0 4
num_bb_nodes, num_subtour_constrs, num_conn_comp = 2 1 3
num_bb_nodes, num_subtour_constrs, num_conn_comp = 4 1 3
num_bb_nodes, num_subtour_constrs, num_conn_comp = 8 1 3
num_bb_nodes, num_subtour_constrs, num_conn_comp = 16 2 3
num_bb_nodes, num_subtour_constrs, num_conn_comp = 32 3 2
num_bb_nodes, num_subtour_constrs, num_conn_comp = 64 3 2
Found a solution with this many subtours: 4
Specifically, they are:
[0, 7, 15, 1, 2, 3]
[1, 2, 3]
[0, 1, 2, 3, 7]
[4, 5, 6, 8, 9, 10, 14]
S_family =  [[0, 7, 15, 1, 2, 3], [1, 2, 3], [0, 1, 2, 3, 7], [4, 5, 6, 8,

Now, we print the table as reported in the paper

In [27]:
# Create a dataframe out of the dictionary out
df = pd.DataFrame(out.items(), columns=["instance", "S_min"])
# Print the dataframe
df

,instance,S_min
0,burma14,2
1,ulysses16,4
2,gr17,5
3,ulysses22,5
4,fri26,4
5,bayg29,4
6,bays29,5
7,swiss42,3
8,dantzig42,4


## HardTSPLIB instances

HardTSPLIB is made of instances generated both at random and starting from instances of the TSPLIB. Unfortunately, we were not able to run the algorithm on the instances generated from TSPLIB. We will only run the algorithm on the instances that are feasible to run on our machine, namely, small random instances. If you want to test other instances, just add them to the `hardtsplib_instances` list at position 0.

In [28]:
hardtsplib_instances = [("10007_hard", 10), ("10010_hard", 10), ("10008_hard", 10), ("10001_hard", 10), ("11675_hard", 11), ("12290_hard", 12), ("14850_hard", 14), ("15005_hard", 15), ("15002_hard", 15), ("15007_hard", 15), ("16038_hard", 16)]

In [29]:
# Store the values in a dictionary
out = {}

In [ ]:
for instance_name, n in hardtsplib_instances:
    # Parse the instance
    G = from_tsplib_file_to_graph("./data/" + instance_name)
    print("******* Instance:", instance_name, "*******")
    S_family, S_num = ialg(G, verbose=True)
    out[instance_name] = S_num

******* Instance: 10007_hard *******
num_bb_nodes, num_subtour_constrs, num_conn_comp = 1 0 2
num_bb_nodes, num_subtour_constrs, num_conn_comp = 2 1 2
num_bb_nodes, num_subtour_constrs, num_conn_comp = 4 3 2
Found a solution with this many subtours: 6
Specifically, they are:
[0, 9, 3, 7]
[8, 1, 2, 6]
[8, 1, 6]
[0, 9, 7]
[8, 2, 6]
[0, 9, 3]
S_family =  [[0, 9, 3, 7], [8, 1, 2, 6], [8, 1, 6], [0, 9, 7], [8, 2, 6], [0, 9, 3]]
******* Instance: 10010_hard *******
num_bb_nodes, num_subtour_constrs, num_conn_comp = 1 0 2
num_bb_nodes, num_subtour_constrs, num_conn_comp = 2 1 2
num_bb_nodes, num_subtour_constrs, num_conn_comp = 4 3 2
Found a solution with this many subtours: 7
Specifically, they are:
[2, 4, 5]
[0, 3, 6, 7]
[0, 3, 6, 7, 8]
[3, 6, 7]
[8, 6, 7]
[8, 3, 6, 7]
[0, 3, 7]
S_family =  [[2, 4, 5], [0, 3, 6, 7], [0, 3, 6, 7, 8], [3, 6, 7], [8, 6, 7], [8, 3, 6, 7], [0, 3, 7]]
******* Instance: 10008_hard *******
num_bb_nodes, num_subtour_constrs, num_conn_comp = 1 0 2
num_bb_nodes, num_s

In [ ]:
# Create a dataframe out of the dictionary out
df = pd.DataFrame(out.items(), columns=["instance", "S_min"])
# Print the dataframe
df